# 기계학습개론 과제4
응용통계학과 </p>
32197453 이동현

In [119]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from tensorflow.keras.layers import Dense, Dropout, Embedding, concatenate, Flatten, Input
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from simpledbf import Dbf5
from keras.models import Sequential, Model

In [120]:
def scores(y_test, y_pred):
    RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
    MAE = mean_absolute_error(y_test, y_pred)
    print('RMSE: {0} \n MAE : {1:.3f}'.format(RMSE, MAE))

## 1. Problem 1 - Pitt

In [121]:
path = "C:/Users/32197453/Desktop/교과/4학년 2학기/기계학습이론/과제/기계학습4/Pitt/"
file_list = os.listdir(path)
dbf_files = []
def find_dbf(files):
    for data in files:
        if '.dbf' in data:
            dbf_files.append(data)
find_dbf(file_list)
#dbf 파일만 추출하여 목록 만들기
print(dbf_files)

for i in dbf_files:
    globals()[i.split('.')[0]] = Dbf5(f'{path}/{i}').to_dataframe()
#목록에 해당하는 dbf 파일 불러온 후 데이터프레임으로 저장

['dat0001.dbf', 'dat0102.dbf', 'dat0206.dbf', 'dat0608.dbf']


In [122]:
#seed 고정
tf.random.set_seed(7453)

#결측치 있는 변수 추출
for i in dbf_files : 
    df = eval(i.replace(".dbf",""))
    print('{0}의 결측치 존재 변수'.format(i.replace(".dbf","")),df.isnull().sum()[df.isnull().sum()!=0])

dat0001의 결측치 존재 변수 Series([], dtype: int64)
dat0102의 결측치 존재 변수 Series([], dtype: int64)
dat0206의 결측치 존재 변수 Series([], dtype: int64)
dat0608의 결측치 존재 변수 Series([], dtype: int64)


### 1) 일반적 모형

In [123]:
scaler = MinMaxScaler()

X_train1_unsc = pd.concat((dat0001, dat0102, dat0206), axis=0).drop('PRICE', axis=1)
y_train1 = pd.concat((dat0001, dat0102, dat0206), axis=0)['PRICE'].astype('int')
X_test1_unsc = dat0608.drop('PRICE', axis=1)
y_test1 = dat0608['PRICE'].astype('int')

X_train1 = scaler.fit_transform(X_train1_unsc)
X_test1 = scaler.fit_transform(X_test1_unsc)


print("Dimension of X_train :", X_train1.shape)
print("Dimension of y_train :", y_train1.shape)
print("Dimension of X_test :", X_test1.shape)
print("Dimension of y_test :", y_test1.shape)

Dimension of X_train : (4821, 21)
Dimension of y_train : (4821,)
Dimension of X_test : (1145, 21)
Dimension of y_test : (1145,)


#### SVM

In [124]:
svm_param = [{"C" : [1, 10,100], 
                "gamma" : [0.1, 1, 5] }]
svm_rnd = RandomizedSearchCV(estimator = SVR(), param_distributions= svm_param,   n_jobs=-1, verbose = 0)
svm_rnd.fit(X_train1, y_train1)

C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=SVR(), n_jobs=-1,
                   param_distributions=[{'C': [1, 10, 100],
                                         'gamma': [0.1, 1, 5]}])

In [125]:
print("최고 평균 정확도 : {0:.6f}".format(svm_rnd.best_score_))
print("최고의 파라미터 : ", svm_rnd.best_params_)
model_G_svm = svm_rnd.best_estimator_
y_pred_G_svm = model_G_svm.predict(X_test1)

최고 평균 정확도 : 0.732800
최고의 파라미터 :  {'gamma': 0.1, 'C': 100}


In [126]:
scores(y_test1, y_pred_G_svm)

RMSE: 92.34123330973253 
 MAE : 27.561


#### Random Forest

In [127]:
rf_param = {'n_estimators' : [100, 200, 400],
            'max_depth' : [4, 8, 16]}
rf_rnd = RandomizedSearchCV(estimator = RandomForestRegressor(), param_distributions= rf_param,   n_jobs=-1, verbose = 0)
rf_rnd.fit(X_train1, y_train1)

C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(estimator=RandomForestRegressor(), n_jobs=-1,
                   param_distributions={'max_depth': [4, 8, 16],
                                        'n_estimators': [100, 200, 400]})

In [128]:
print("최고 평균 정확도 : {0:.6f}".format(rf_rnd.best_score_))
print("최고의 파라미터 : ", rf_rnd.best_params_)
model_G_rf = rf_rnd.best_estimator_
y_pred_G_rf = model_G_rf.predict(X_test1)

최고 평균 정확도 : 0.733308
최고의 파라미터 :  {'n_estimators': 200, 'max_depth': 8}


In [129]:
scores(y_test1, y_pred_G_rf)

RMSE: 92.05015290386002 
 MAE : 34.991


#### XGBoost

In [130]:
xgb_param = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}
xgb_rnd = RandomizedSearchCV(estimator = XGBRegressor(), param_distributions= xgb_param,   n_jobs=-1, verbose = 0)
xgb_rnd.fit(X_train1, y_train1)

RandomizedSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, gpu_id=None,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=None,...
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=None,
                                          max_leaves=None,
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          n_estimators=100, n_jobs=None,
                                          num_parallel_tree=None,
                                          predictor=None, random_state=None, ...),
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.1, 0.3],
                                        'max_depth': [4, 8, 12],
                                        'n_estimators': [100, 200, 400]})

In [131]:
print("최고 평균 정확도 : {0:.6f}".format(xgb_rnd.best_score_))
print("최고의 파라미터 : ", xgb_rnd.best_params_)
model_G_xgb = xgb_rnd.best_estimator_
y_pred_G_xgb = model_G_xgb.predict(X_test1)

최고 평균 정확도 : 0.722230
최고의 파라미터 :  {'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.1}


In [132]:
scores(y_test1, y_pred_G_xgb)

RMSE: 94.39166255787589 
 MAE : 33.203


#### LightGBM

In [134]:
lgbm_param = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}

lgbm_rnd = RandomizedSearchCV(estimator=LGBMRegressor(), param_distributions= lgbm_param,   n_jobs= -1, verbose = 0)
lgbm_rnd.fit(X_train1, np.ravel(y_train1))



RandomizedSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.1, 0.3],
                                        'max_depth': [4, 8, 12],
                                        'n_estimators': [100, 200, 400]})

In [135]:
print("최고 평균 정확도 : {0:.6f}".format(lgbm_rnd.best_score_))
print("최고의 파라미터 : ", lgbm_rnd.best_params_)
model_G_lgbm = lgbm_rnd.best_estimator_
y_pred_G_lgbm = model_G_lgbm.predict(X_test1)

최고 평균 정확도 : 0.727018
최고의 파라미터 :  {'n_estimators': 100, 'max_depth': 4, 'learning_rate': 0.1}


In [136]:
scores(y_test1, y_pred_G_lgbm)

RMSE: 92.5423520818397 
 MAE : 31.460


#### Neural Network

In [137]:
n_steps_in, n_steps_out = 21, 1

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps_in))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit([X_train1], y_train1, epochs=2000, verbose=0)

# demonstrate prediction
y_pred_NN = model.predict([X_test1], verbose=0)

In [138]:
scores(np.array(y_test1), y_pred_NN)

RMSE: 96.07154446080871 
 MAE : 30.933


평가 지표중에서 RMSE에는 Random Forest가 가장 낮은 92.05를 나타냈다. MAE에서는 SVM이 27.561로 가장 낮은 값을 보였다.

### 2) Continual Learning

In [139]:
scaler = MinMaxScaler()

X_train_C1 = scaler.fit_transform(pd.DataFrame(dat0001).drop('PRICE', axis=1))
y_train_C1 = np.array(dat0001['PRICE'].astype('int'))

X_train_C2 = scaler.fit_transform(pd.DataFrame(dat0102).drop('PRICE', axis=1))
y_train_C2 = np.array(dat0102['PRICE'].astype('int'))

X_train_C3 = scaler.fit_transform(pd.DataFrame(dat0206).drop('PRICE', axis=1))
y_train_C3 = np.array(dat0206['PRICE'].astype('int'))

X_test_C3 = scaler.fit_transform(pd.DataFrame(dat0608).drop('PRICE', axis=1))
y_test_C3 = np.array(dat0608['PRICE'].astype('int'))

print("Dimension of X_train :", X_train1.shape)
print("Dimension of y_train :", y_train1.shape)
print("Dimension of X_test :", X_test1.shape)
print("Dimension of y_test :", y_test1.shape)

Dimension of X_train : (4821, 21)
Dimension of y_train : (4821,)
Dimension of X_test : (1145, 21)
Dimension of y_test : (1145,)


#### Random Forest

In [140]:
rf_param = {'n_estimators' : [100, 200, 400],
            'max_depth' : [4, 8, 16],
            'warm_start' : [True]}
rf_rnd = RandomizedSearchCV(estimator = RandomForestRegressor(), param_distributions= rf_param,   n_jobs=-1, verbose = 0)

rf_rnd.fit(X_train_C1, y_train_C1)
rf_model_C1 = rf_rnd.best_estimator_

rf_rnd.fit(X_train_C2, y_train_C2)
rf_model_C2 = rf_rnd.best_estimator_

rf_rnd.fit(X_train_C3, y_train_C3)
rf_model_C3 = rf_rnd.best_estimator_

rf_rnd.fit(X_train_C3, y_train_C3)
model_C_rf = rf_rnd.best_estimator_

print("warning 출력 생략")


C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Runnin

warning 출력 생략


In [141]:
y_pred_C_rf = model_C_rf.predict(X_test1)
scores(y_test1, y_pred_C_rf)

RMSE: 109.08366632715035 
 MAE : 54.265


#### XGBoost

In [143]:
xgb_param = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}
xgb_rnd = RandomizedSearchCV(estimator = XGBRegressor(), param_distributions= xgb_param, n_jobs=-1, verbose = 0)

xgb_rnd.fit(X_train_C1, y_train_C1)
xgb_model_C1 = xgb_rnd.best_estimator_

xgb_rnd.fit(X_train_C2, y_train_C2, xgb_model=xgb_model_C1)
xgb_model_C2 = xgb_rnd.best_estimator_

xgb_rnd.fit(X_train_C3, y_train_C3, xgb_model= xgb_model_C2)
model_C_xgb = xgb_rnd.best_estimator_

In [144]:
y_pred_C_xgb = model_C_xgb.predict(X_test1)
scores(y_test1, y_pred_C_xgb)

RMSE: 102.7141744800217 
 MAE : 43.144


#### LightGBM

In [146]:
lgbm_param = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}

lgbm_rnd = RandomizedSearchCV(estimator=LGBMRegressor(), param_distributions= lgbm_param, n_jobs= -1, verbose = 0)
lgbm_rnd.fit(X_train1, np.ravel(y_train1))

lgbm_rnd.fit(X_train_C1, y_train_C1)
lgbm_model_C1 = lgbm_rnd.best_estimator_

lgbm_rnd.fit(X_train_C2, y_train_C2, init_model=lgbm_model_C1)
lgbm_model_C2 = lgbm_rnd.best_estimator_

lgbm_rnd.fit(X_train_C3, y_train_C3, init_model=lgbm_model_C2)
model_C_lgbm = lgbm_rnd.best_estimator_

In [147]:
y_pred_C_lgbm = model_C_lgbm.predict(X_test1)
scores(y_test1, y_pred_C_lgbm)

RMSE: 101.03628537291952 
 MAE : 48.051


Continual Learning을 시행했을 때, RMSE에서는 LightGBM이 101.036으로 가장 낮은 값을, MAE에서는 XGBoost가 43.144로 가장 낮은 값을 보였다. 전체를 한번에 학습했을때보다 Continual Learning을 했을때 오차값이 약간 더 크게 나타났다.

## 2. Problem 2 - Carteret

In [148]:
path = "C:/Users/32197453/Desktop/교과/4학년 2학기/기계학습이론/과제/기계학습4/Carteret/"
df2 = pd.read_excel(path + 'house_short.xlsx')
print("--------데이터셋 차원 확인")
print(df2.shape)
print("--------결측값 있는 변수 확인")
missval = df2.isnull().sum()[df2.isnull().sum()!=0]
print(missval)
missval_name = missval.index #결측값 있는 변수 이름 벡터로 추출

--------데이터셋 차원 확인
(3106, 30)
--------결측값 있는 변수 확인
Series([], dtype: int64)


In [149]:
X2_unsc = df2.drop(["SALE_PRICE", "DDATE"], axis = 1)
X2 = scaler.fit_transform(X2)
Y2 = np.array(df2['SALE_PRICE'])

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, Y2, test_size=1033, random_state=1234)

print("Dimension of X_train :", X_train2.shape, "\n Dimension of y_train :", y_train2.shape, "\n Dimension of X_test :", X_test2.shape, "\n Dimension of y_test :", y_test2.shape)

Dimension of X_train : (2073, 28) 
 Dimension of y_train : (2073,) 
 Dimension of X_test : (1033, 28) 
 Dimension of y_test : (1033,)


#### SVM

In [150]:
svm_param = [{"C" : [1,10,50,100], 
                "gamma" : [0.1, 1, 5, 10] }]
svm_rnd = RandomizedSearchCV(estimator = SVR(), param_distributions= svm_param, n_jobs=-1, verbose=0)
svm_rnd.fit(X_train2, y_train2)

print("최고 평균 정확도 : {0:.6f}".format(svm_rnd.best_score_))
print("최고의 파라미터 : ", svm_rnd.best_params_)
model_svm2 = svm_rnd.best_estimator_
y_pred_svm2 = model_svm2.predict(X_test2)

print(scores(y_test2, y_pred_svm2))

최고 평균 정확도 : -0.050128
최고의 파라미터 :  {'gamma': 1, 'C': 100}
RMSE: 112881.2177063189 
 MAE : 67878.765
None


#### Random Forest

In [152]:
rf_param = {'n_estimators' : [100, 200, 400],
            'max_depth' : [4, 8, 16]}
rf_rnd = RandomizedSearchCV(estimator = RandomForestRegressor(), param_distributions= rf_param, n_jobs=-1, verbose=0)
rf_rnd.fit(X_train2, y_train2)

print("최고 평균 정확도 : {0:.6f}".format(rf_rnd.best_score_))
print("최고의 파라미터 : ", rf_rnd.best_params_)
model_rf2 = rf_rnd.best_estimator_
y_pred_rf2 = model_rf2.predict(X_test2)

print(scores(y_test2, y_pred_rf2))

C:\Users\32197453\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\model_selection\_search.py:306: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


최고 평균 정확도 : 0.606077
최고의 파라미터 :  {'n_estimators': 400, 'max_depth': 16}
RMSE: 66342.44098376475 
 MAE : 37815.618
None


#### XGBoost

In [153]:
xgb_param = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}
xgb_rnd = RandomizedSearchCV(estimator = XGBRegressor(), param_distributions= xgb_param, n_jobs=-1, verbose=0)
xgb_rnd.fit(X_train2, y_train2)

print("최고 평균 정확도 : {0:.6f}".format(xgb_rnd.best_score_))
print("최고의 파라미터 : ", xgb_rnd.best_params_)
model_xgb2 = xgb_rnd.best_estimator_
y_pred_xgb2 = model_xgb2.predict(X_test2)

print(scores(y_test2, y_pred_xgb2))

최고 평균 정확도 : 0.639018
최고의 파라미터 :  {'n_estimators': 200, 'max_depth': 4, 'learning_rate': 0.1}
RMSE: 62666.94449821618 
 MAE : 37497.385
None


#### LigthGBM

In [157]:
lgbm_param = {
    'n_estimators' : [100, 200, 400],
    'learning_rate' : [0.01, 0.1, 0.3],
    'max_depth' : [4, 8, 12],
}

lgbm_rnd = RandomizedSearchCV(estimator=LGBMRegressor(), param_distributions= lgbm_param, n_jobs= -1, verbose = 0)
lgbm_rnd.fit(X_train2, y_train2)

print("최고 평균 정확도 : {0:.6f}".format(lgbm_rnd.best_score_))
print("최고의 파라미터 : ", lgbm_rnd.best_params_)
model_lgbm2 = lgbm_rnd.best_estimator_
y_pred_lgbm2 = model_lgbm2.predict(X_test2)

print(scores(y_test2, y_pred_lgbm2))

최고 평균 정확도 : 0.607742
최고의 파라미터 :  {'n_estimators': 400, 'max_depth': 4, 'learning_rate': 0.1}
RMSE: 61782.95021519949 
 MAE : 37781.716
None


#### Neural Network

In [158]:
n_steps_in, n_steps_out = 28, 1

model = Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps_in))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

# fit model
model.fit([X_train2], y_train2, epochs=2000, verbose=0)

# demonstrate prediction
y_pred_NN2 = model.predict([X_test2], verbose=0)

scores(y_test2, y_pred_NN2)

RMSE: 86826.14556643121 
 MAE : 52671.171


Carteret의 집값 데이터에서는 LightGBM이 61782.950로 가장 낮은 RMSE를 보였고, XGBoost가 37497.385로 가장 낮은 MAE를 보였다.